In [ ]:
import sys  
sys.path.insert(1, '..')
sys.path.insert(2, '../modules/')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
from glob import glob
from datetime import timedelta
from modules import convert_datetime
import dataconfig

In [ ]:
merged_df_list = []
glob_str = f'{dataconfig.DATA_DIR_ZERO_CROSS}/*zero_cross_df.pickle'
for file in glob(glob_str):
    input_data = pickle.load(open(file, 'rb'))

    merged_df_list.append(input_data)

merged_zerocross_df = pd.concat(merged_df_list)

In [ ]:
hek_flares = pickle.load(open('/data/padialjr/jorge-helio/data_products/hek_flare_db.pickle', 'rb'))

In [ ]:
flare_class_dict_query = {'A': 1e-8, 'B': 1e-7, 'C': 1e-6, 'M': 1e-5, 'X': 1e-4}
teams = {'ALEXIS': 1, 'SWPC': 2, 'SolarSoft':4}  
# tuples are (ALEXIS, SWPC, solar_soft)    
id_dict = {'7':(1,1,1), '6': (0,1,1), '5': (1,0,1),'4': (0,0,1),'3': (1,1,0),'2': (0,1,0),'1': (1,0,0)}

In [ ]:
goes_flares = hek_flares[(hek_flares['id_team'] == 'SWPC')& (hek_flares['goes_class'] > 'C')].reset_index(drop= True)

solarsoft_flares = hek_flares[(hek_flares['id_team'] == 'SolarSoft') & (hek_flares['goes_class'] > 'C')].reset_index(drop= True)

zero_cross = merged_zerocross_df[(merged_zerocross_df['resampled_value'] > flare_class_dict_query['C'])].reset_index(drop= True)


In [ ]:
#define our zero crossing df

our_flares = zero_cross[['zerocross_date_time']]

our_flares['id_team'] = 'ALEXIS'

only_duplicated_zero = our_flares[(our_flares.duplicated())]

zero_flares = our_flares.rename(columns = {'zerocross_date_time': 'peak_time'})

In [ ]:

# define SWPC Flares

goes_flares = pd.DataFrame(goes_flares.peak_time, columns = ['peak_time'])

goes_flares['id_team'] = 'SWPC'

In [ ]:
# define solarsoft flares

solarsoft_flares = pd.DataFrame(solarsoft_flares.peak_time, columns = ['peak_time'])

solarsoft_flares['id_team'] = 'SolarSoft'


In [ ]:
all_flares_not_masked = pd.concat([zero_flares , goes_flares,  solarsoft_flares], ignore_index=True)

date_limit_low, date_limit_high = pd.Timestamp('2010/5/1T00:00:00', tz = 'utc'),pd.Timestamp('2020/3/4T00:00:00', tz = 'utc')

all_flares = all_flares_not_masked[(all_flares_not_masked.peak_time >= date_limit_low) & (all_flares_not_masked.peak_time <= date_limit_high)].sort_values(by = 'peak_time').reset_index(drop = True)

In [ ]:
minutes_per_agg = 10

j = 0 # grab first datetime in the list
flares_list = [] # keep track of info from loop

while j < len(all_flares.peak_time):
    
    datetime = all_flares.peak_time.iloc[j] # grab j-th datetime
        
    fwd = datetime + timedelta(minutes = minutes_per_agg) # choose fwd time range
     
    mask = all_flares[(all_flares.peak_time < fwd) & (all_flares.peak_time >= datetime)] # return df with all entries from current datetime to fwd time
    
    this_flare_datetime_list = []
    
    this_flare_identification = []
    
    for group in teams: 
        
        group_mask = mask[(mask.id_team) == group] #filter for all elements of mask that are returned by each individual group
        
        number_of_entries = len(group_mask)
        
        if number_of_entries != 0:
        
            mean_datetime = np.mean(group_mask.peak_time)
            
            this_flare_datetime_list.append({'datetime': mean_datetime})
            
            this_flare_identification.append(teams[group])
                   
        
    sum_of_ids = np.sum(this_flare_identification)
    
    flare_id_tuple = id_dict[str(sum_of_ids)]
    
    datetime_df = pd.DataFrame(this_flare_datetime_list)
    
    merged_datetime = np.mean(datetime_df.datetime)
            
    flares_list.append({'merged_datetime': merged_datetime,
                        'id_tuple': flare_id_tuple})
    
    k = len(mask) # go k amount fwd in the list of datetimes

    j = j + k

In [ ]:
flare_list_df = pd.DataFrame(flares_list)
flare_list_df

In [ ]:
plot_this = []
for name,group in pd.DataFrame(flares_list).groupby('id_tuple'):
    
    plot_this.append({'id_tuple': name, 
                     'counts': len(group)})

In [ ]:
stat_df = pd.DataFrame(plot_this)

stat_df


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (20,10))
ax = fig.add_subplot(111)


# Example data
id_tuple = stat_df.id_tuple.to_list()
y_pos = np.arange(len(id_tuple))
counts = stat_df.counts.to_list()
# error = np.random.rand(len(people))



hbars = ax.barh(y_pos, counts, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(id_tuple)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('counts')
ax.set_ylabel('id_tuple == (jonas_lab, SWPC, solarsoft)')
ax.set_title('id_tuple == (jonas_lab, SWPC, solarsoft) @ {} minute fwd time'.format(minutes_per_agg))

for bar in hbars:
    print(bar.get_y())
    ax.text(bar.get_width(), bar.get_y(), s = bar.get_width(), va = 'center')

ax.set_xscale('log')